<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/langgraph_memory_base_email_assistant_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental" "aiosqlite"

In [15]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
os.environ["TAVILY_API_KEY"] = userdata.get('tavily_api_key')

# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    verbose=True
)


StepI. Emails classification

In [17]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team"
}
prompt_instructions = {
  "triage_rules": {
      "ignore": "Marketing newsletters, spam emails",
      "notify": "Team member out sick, build system notifications.",
      "respond": "Direct questions from team members, meeting requests."
  },
  "agent_instructions": "Use these tools when appropriate to help"
}

In [20]:
# Define output structure
from pydantic import BaseModel, Field
from typing import Literal
from langchain.chat_models import init_chat_model

class Router(BaseModel):
  """Analyze the unread email and route it according to its content"""

  reasoning: str = Field(
      description="Step-by-step reasoning behind the classification"
  )
  classification: Literal["ignore", "respond", "notify"] = Field(
      description="The classification of an email: 'ignore' for irrelavant emails, \
      'notify' for important information that doesn't need a response,\
      'respond' for emails that need a reply"
  )
llm_router = llm.with_structured_output(Router)

In [35]:
# Define prompts schemas
triage_system_prompt = """
<Role>
You are {full_name}'s executive assistant designed to help manage incoming emails. You are \
a top-notch executive assistant who cares about {name} performing as well as possible.
</Role>

<Background>
{user_profile_background}
</Background>

<Instructions>

{name} gets lots of emails. Your job is to categorize each email into on of three\
categories:
1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but\
doesn't require a response
3. RESPOND - Emails that require a direct response from {name}

Classify the below email into one of these categories.

</Instructions>

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email\
response. For these, you should notify {name} (using the 'notofy' response). Examples \
of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >
{examples}
</ Few shot examples >
"""

triage_user_prompt = """
Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}

"""

agent_system_prompt = """
<Role>
You are {full_name}'s executive assistant designed to help manage incoming emails. You are \
a top-notch executive assistant who cares about {name} performing as well as possible.
</Role>

<Tools>
You have access to the following tools to help mamage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_date) - Schedule calendar meetings
3. check_calender_availablity(day) - Check calender available time slots for a given day.
</Tools>

< Instructions >
{instructions}
</ Instructions >

"""

In [24]:
# Define prompts
system_prompt = triage_system_prompt.format(
    full_name=profile['full_name'],
    name=profile['name'],
    user_profile_background=profile['user_profile_background'],
    examples = None,
    triage_no = prompt_instructions['triage_rules']['ignore'],
    triage_notify = prompt_instructions['triage_rules']['notify'],
    triage_email = prompt_instructions['triage_rules']['respond']
)

In [25]:
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "Jone Doe<John.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
    Hi John,

    I was reviewing the API documentation for the new authentication service but got some questions for several paths.

    Specifically, I'm looking at:
    - /auth/refresh
    - /auth/validate

    Thanks,
    Alice"""
}

In [26]:
user_prompt = triage_user_prompt.format(
    author=email['from'],
    to=email['to'],
    subject=email['subject'],
    email_thread=email['body']
)

In [27]:
result = llm_router.invoke([
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
])
print(result)

reasoning='The email is a direct question from a team member (Alice Smith) regarding API documentation. According to the rules, direct questions from team members require a response.' classification='respond'


Step II. Create response agent with tools to handle emails with response required

In [31]:
# Define tools (mocked behaviors)
from langchain_core.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
  """Write and send an email"""
  # Placeholder response - real app
  return f"Email send to: {to}\nSubject: {subject}\nBody: {content}"
@tool
def schedule_meetings(
    attendees: list[str],
    subject: str,
    duration_minutes: int,
    preferred_date: str
) -> str:
  """Schedule a calendar meeting with a list of attendees"""
  # Placeholder response - real app
  return f"Meeting '{subject}' scheduled for {preferred_date}"
@tool
def check_calender_availablity(day:str) -> str:
  """Check calender availability for a given day. """
  # Placeholder response - real app...
  return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"




In [36]:
def create_prompt(state):
  return [
      {
          "role": "system",
          "content": agent_system_prompt.format(
              instructions = prompt_instructions['agent_instructions'],
              **profile
          )
      }
  ] + state['messages']

In [37]:
# Create the agent
from langgraph.prebuilt import create_react_agent
tools = [write_email, schedule_meetings, check_calender_availablity]
agent = create_react_agent(
    llm,
    tools,
    prompt = create_prompt,
)

/tmp/ipython-input-526044832.py:4: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [38]:
response = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "what's my availability for Tuesday?"
    }]
})
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

[{'type': 'text', 'text': 'You are available at 9:00 AM, 2:00 PM, and 4:00 PM on Tuesday.', 'extras': {'signature': 'CqUDAb4+9vvVthH5LCbKAm/+pxoPZxHL67lezyaBaxeSCUhm5t74QLvkkyrazdj39c9tn5fc63kCMUZkbmbdkOr18CICOJcP49PlX1WbIOzA9zAIzVaed6BYc5CtqaNRWStyVms0aVBe0jJWH4PIKPLNTERi1QRiq1V3D7/VHClXABO82YbLqfA550X91wUn7XWdxXsIRhxTDVIEmP5hpPfPyy7NfxH3kd4JycnFoBhKFim8YTIMrZ7dFzLlSiOsCle5DQl1hL9HUvFprcN+y0ShDSBXOTAfI74S7zVk26OJx1CdA0ydnuP7vkE9F46XA4ClQ9ZOao6eWQuWdtO/8C+DFCNWsFS+upwyQ0UsBQduJMU/Qoz6cCPXB2hIGiW8gfSZ8dvXcOlc6Lv1wtDblm1xW/yf4BtHVPd1R4LPpE6ONJszGZhKiQ7J68BA2eRYhYe9mcFYMpptp2nOWXBUVkFt9i6kErhcMUOdsRvp/QhbqBc9ga3AHMkCLaaoXoL+VeVAwtFW0SPIwbeo7HbsDm/VoiYones65RV1W96uSHC0VLH1YWlk/g=='}}]


Step III. Create the whole email agent

In [40]:
# Define state
from langgraph.graph import add_messages
from typing import TypedDict, Annotated

class State(TypedDict):
  email_input: dict
  messages: Annotated[list, add_messages]

In [53]:
# Define the whole agent
from langgraph.graph import END, START, StateGraph
from langgraph.types import Command
from typing import Literal # Corrected import for Literal

def triage_router(state: State) -> dict: # Changed function name and return type
  author = state['email_input']['from']
  to = state['email_input']['to']
  subject = state['email_input']['subject']
  email_thread = state['email_input']['body']

  system_prompt = triage_system_prompt.format(
    full_name=profile['full_name'],
    name=profile['name'],
    user_profile_background=profile['user_profile_background'],
    examples = None,
    triage_no = prompt_instructions['triage_rules']['ignore'],
    triage_notify = prompt_instructions['triage_rules']['notify'],
    triage_email = prompt_instructions['triage_rules']['respond']
  )
  user_prompt = triage_user_prompt.format(
    author=author,
    to=to,
    subject=subject,
    email_thread=email_thread
  )
  result = llm_router.invoke(
      [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt},
      ]
  )

  classification_decision = result.classification
  print(f"Classification: {classification_decision}")

  if classification_decision == "respond":
    update = {
        "messages" : [
        {
            "role": "user",
            "content": f"Respond to the email {state['email_input']}"
        }]
    }
    goto = "response_agent"
  elif classification_decision == "ignore" or classification_decision == "notify":
    # No messages update for ignore/notify in this flow, just the classification result
    goto = END
    update = None
  else:
    raise ValueError(f"Invalid classification: {classification_decision}")

  return Command(goto=goto, update=update)

email_agent = StateGraph(State)
email_agent.add_node("triage_router_node", triage_router) # Node where classification happens
email_agent.add_node("response_agent", agent) # The agent node for responding

# Define the entry point
email_agent.add_edge(START, "triage_router_node")

# Add conditional edges from the triage_router_node
'''
email_agent.add_conditional_edges(
    "triage_router_node",  # The node whose output determines the next step
    route_classification,  # The function that routes based on state
    {
        "response_agent": "response_agent", # If route_classification returns "response_agent", go here
        END: END               # If route_classification returns END, end the graph
    }
)
'''
email_agent = email_agent.compile()

In [54]:
response = email_agent.invoke({
    "email_input": email
})

Classification: respond


In [55]:
for m in response["messages"]:
  m.pretty_print()

================================ Human Message =================================

Respond to the email {'from': 'Alice Smith <alice.smith@company.com>', 'to': 'Jone Doe<John.doe@company.com>', 'subject': 'Quick question about API documentation', 'body': "\n    Hi John,\n    \n    I was reviewing the API documentation for the new authentication service but got some questions for several paths.\n\n    Specifically, I'm looking at:\n    - /auth/refresh\n    - /auth/validate\n\n    Thanks,\n    Alice"}
================================== Ai Message ==================================
Tool Calls:
  write_email (dc965e3c-097b-4523-a14d-8e90bb2488f5)
 Call ID: dc965e3c-097b-4523-a14d-8e90bb2488f5
  Args:
    content: Hi Alice,

Thanks for reaching out. John would be happy to discuss your questions regarding the API documentation for /auth/refresh and /auth/validate.

Would you like to schedule a quick meeting to go over them? Please let us know your availability, and we can find a suitable time